<a href="https://colab.research.google.com/github/Alimbj/Birds_Classification/blob/main/Cattle_genotyping_clasterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Анализ генотипических данных КРС

Шаг 1. Linux. Преобразование исходных файлов .ped и .map в бинарный формат PLINK


In [ ]:
for dir in 2 3 4 5 6 7; do
  do echo "Processing folder $dir";
  cd $dir
  basefile = $(basename $(cattle_*.ped) .ped);
  plink --file "$basefile" --cow --make-bed --out "plinkdata_${dir}"
  cd ..
done


Шаг 2. Фильтрация по общим SNP и создание подмножеств

In [ ]:
for i in 2 3 4 5 6 7; do
  plink --bfile plinkdata_${i} --cow --extract common_snps.txt --make-bed --out filtered_${i}
done


Шаг 3. Слияние подмножеств

In [ ]:
cp filtered_2.bed merged.bed
cp filtered_2.bim merged.bim
cp filtered_2.fam merged.fam


# Затем последователно сливать остальные

for i in 3 4 5 6 7; do
  plink --bfile merged \
        --bmerge filtered_${i}.bed filtered_${i}.bim filtered_${i}.fam \
        --cow \
        --make-bed \
        --out merged_temp

  if [ -f merged_temp.bed ]; then
    mv merged_temp.bed merged.bed
    mv merged_temp.bim merged.bim
    mv merged_temp.fam merged.fam
  else
    echo "Merge with filtered_${i} failed"
    break
  fi
done


Шаг 4. Проведение PCA с помощью PLINK

In [ ]:
plink --bfile merged --pca 10 --out merged_pca


### 2. Визуализация и кластеризация в Python (Google Colab)

Шаг 1. Загрузка результатов PCA и метаданных, визуализация PCA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap

fam = pd.read_csv("merged.fam", delim_whitespace=True, header=None)
fam.columns = ["FID", "IID", "FatherID", "MotherID", "Sex", "Phenotype"]

# Загрузить eigenvec файл (пропустим первую колонку с FID и вторую с IID)
df = pd.read_csv("merged_pca.eigenvec", delim_whitespace=True, header=None)

# Добавим имена колонок
df.columns = ["FID", "IID"] + [f"PC{i}" for i in range(1, df.shape[1] - 1)]

df = pd.merge(df, fam[["FID", "IID"]], on=["FID", "IID"], how="left")

plt.figure(figsize=(10, 7))
sns.scatterplot(data=df, x="PC1", y="PC2", hue="FID", palette="coolwarm", s=60)
plt.title("PCA of Cattle Genotypes by Breed (FID)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df2 = pd.read_csv('/content/merged_pca.eigenval')
df2

Шаг 3. Визуализация с UMAP

In [ ]:
# Вытаскиваем только числовые главные компоненты (например, первые 10 PC)
X = df[[f"PC{i}" for i in range(1, 11)]].values

# Запускаем UMAP (можешь настроить n_neighbors и min_dist по желанию)
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, random_state=42)
embedding = reducer.fit_transform(X)

# Добавляем результаты UMAP в DataFrame
df["UMAP1"] = embedding[:, 0]
df["UMAP2"] = embedding[:, 1]

# Визуализация
plt.figure(figsize=(10, 7))
sns.scatterplot(data=df, x="UMAP1", y="UMAP2", hue="FID", palette="coolwarm", s=60)
plt.title("UMAP projection of Cattle Genotypes by Breed (FID)")
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.grid(True)
plt.tight_layout()
plt.show()

Шаг 4. Кластеризация и визуализация t-SNE

In [ ]:
from sklearn.manifold import TSNE

# Можно применить к тем же 10 PC или к полным данным
X_tsne = df[[f"PC{i}" for i in range(1, 11)]].values

tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_embedding = tsne.fit_transform(X_tsne)

df["tSNE1"] = tsne_embedding[:, 0]
df["tSNE2"] = tsne_embedding[:, 1]

plt.figure(figsize=(10, 7))
sns.scatterplot(data=df, x="tSNE1", y="tSNE2", hue="FID", palette="coolwarm", s=60)
plt.title("t-SNE projection of Cattle Genotypes by Breed (FID)")
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.grid(True)
plt.tight_layout()
plt.show()


Шаг 5. Кластеризация DBSCAN на UMAP

In [ ]:
from sklearn.cluster import DBSCAN

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
embedding = reducer.fit_transform(X)

df["UMAP1"] = embedding[:, 0]
df["UMAP2"] = embedding[:, 1]

# Используем уже полученный embedding из UMAP
X = df[["UMAP1", "UMAP2"]].values

# Настраиваем параметры DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(X)

# Добавим метки кластеров в DataFrame
df["DBSCAN_cluster"] = clusters

# Визуализируем
plt.figure(figsize=(10, 7))
sns.scatterplot(data=df, x="UMAP1", y="UMAP2", hue="DBSCAN_cluster", palette="tab20", s=60)
plt.title("DBSCAN clustering on UMAP projection")
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
